

---



---




<div style="text-align: center; font-size: 30px; font-weight: bold;">
  DETECCIÓN DEL LÍMITE AGUA-TIERRA (DIG - IGN)
</div>



---



---






In [ ]:
import ee
import geemap
import ipywidgets as widgets

from datetime import datetime
import pandas as pd
import requests
from IPython.display import HTML, display
import os
from zmq.constants import NULL
from ipywidgets import Select

import json

In [ ]:
# Utilizar la variable de entorno para inicializar Earth Engine
service_account_json = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
if service_account_json:
    ee.Initialize()
else:
    print("Error: No se encontraron las credenciales de Earth Engine")

In [ ]:
Map = geemap.Map(center=(-40, -64), zoom=4) #crear el mapa inetrcativo con coordenadas especificas
Map.add_basemap('HYBRID')

In [ ]:
etiqueta_mensaje_clasif = widgets.Label(value=' ')

global classified_b
global extencion

classified_b = None
extencion = None

In [ ]:
#..................................................... Cartas
#cartas = ee.FeatureCollection("projects/e-tenglerevelyn/assets/Cartas_50000")
cartas = ee.FeatureCollection("projects/ee-dig-aplicaciones/assets/AguaTierra/Cartas_50000")

# Obtener los valores únicos de la columna "carac"
valoresCarac_ = cartas.aggregate_array("carac").distinct().getInfo()

# Crear etiquetas
#etiqueta = widgets.Label(value='Seleccione una carta 1:50.000')
etiqueta_fecha = widgets.Label(value='Elija el período para la clasificación de las imágenes: ')
etiqueta_linea = widgets.Label(value='____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________')
etiqueta_vacia = widgets.Label(value='                                                    ')



# Crear una lista de valores, incluyendo una opción nula
valoresCarac = [None] + valoresCarac_  # Reemplaza "tus_valores" con tu lista de valores

# Crear la lista desplegable con las opciones
lista_registros = Select(options=valoresCarac, description='Opciones')


# Definir una función para manejar el evento de cambio de selección
def on_value_change(change):
    valor_carta = change['new']
    if valor_carta is not None:  # Evitar el procesamiento si se selecciona la opción nula
        geometria_seleccionada = cartas.filter(ee.Filter.eq("carac", valor_carta))
        Map.addLayer(geometria_seleccionada, {}, 'Geometría seleccionada')

# Registrar la función de manejo del evento de cambio
lista_registros.observe(on_value_change, names='value')

fecha_inicial = datetime(2020, 1, 1)

#..................................................... Período de tiempo
date_picker_inicio = widgets.DatePicker(
    description='Fecha inicial:',
    value=fecha_inicial,  # Establecer la fecha actual como valor inicial
    disabled=False
)

date_picker_fin = widgets.DatePicker(
    description='Fecha final:',
    value=datetime.now().date(),  # Establecer la fecha actual como valor inicial
    disabled=False
)

hbox_fecha= widgets.HBox([date_picker_inicio, date_picker_fin])

#..............................................................

opcion_area = widgets.RadioButtons(
    options=['Dibujar polígono en el mapa', 'Utilizar carta 1:50.000'],
    description='Seleccione la forma de determinar el área de interés: ',
    #layout=widgets.Layout(width='80%')
)

In [ ]:
# MUESTRAS
#muestras = ee.FeatureCollection("projects/ee-gonzalodgign/assets/AguaTierra/LineadeCosta/Muestras_Argentina_Borrador")
muestras = ee.FeatureCollection("projects/ee-dig-aplicaciones/assets/AguaTierra/Muestras_Argentina_Borrador")

# ZONA DE ENTRENAMIENTO
#zona_muestreo = ee.FeatureCollection("projects/ee-tenglerevelyn/assets/AREA_A-T")
zona_muestreo = ee.FeatureCollection("projects/ee-dig-aplicaciones/assets/AguaTierra/AREA_A-T")

In [ ]:
# Colección de imágenes Sentinel 2
s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
    .filterDate('2023-01-01', '2023-03-31') \
    .filterBounds(zona_muestreo) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.2)

s2c_ = s2.reduce(ee.Reducer.median())
s2c = s2c_.clip(zona_muestreo)

In [ ]:
# NDWI
NDWI = s2c.normalizedDifference(['B3_median', 'B8_median']).rename('NDWImedian')

# Stack
stackIslas_a = NDWI

In [ ]:
muestras_1 = muestras.randomColumn('random')

# División de muestras
entrenamiento = muestras_1.filter(ee.Filter.lt('random', 0.7))
validacion = muestras_1.filter(ee.Filter.gt('random', 0.7))

# Entrenar el clasificador utilizando datos de muestras
training = stackIslas_a.sampleRegions(
  collection=entrenamiento,
  properties=['class', 'random'],
  scale=30
)

# Bandas a utilizar en la clasificación
bandas_sel = ['NDWImedian']

# Entrenar clasificador RF
classifier = ee.Classifier.minimumDistance().train(
    features = training,
    classProperty = 'class',
    inputProperties = bandas_sel
)

In [ ]:
def obtenerFecha():
    time_interval = None

    etiqueta_mensaje_clasif.value=''
    # Obtener las fechas seleccionadas
    fecha_inicio = date_picker_inicio.value
    fecha_fin = date_picker_fin.value

    # Convertir las fechas al formato deseado ('YYYY-MM-DD')
    fecha_inicio_str = fecha_inicio.strftime('%Y-%m-%d')
    fecha_fin_str = fecha_fin.strftime('%Y-%m-%d')

    # Convertir las fechas en objetos ee.Date
    fecha_inicio_ee = ee.Date(fecha_inicio_str)
    fecha_fin_ee = ee.Date(fecha_fin_str)

    # Calcular la diferencia en milisegundos
    diferencia = fecha_inicio_ee.difference(fecha_fin_ee, 'days')

    # Verificar si la diferencia es negativa
    if diferencia.getInfo() >= 0:
        etiqueta_mensaje_clasif.value='La fecha final no puede ser mayor o igual que la fecha inicial'
        # Puedes manejar el error de la manera que desees, por ejemplo, mostrar un mensaje de error.
    else:
        # Utilizar las fechas convertidas en el parámetro time_interval
        time_interval = (fecha_inicio_ee, fecha_fin_ee)

        # Retornar el resultado
    return time_interval

def area_extencion():
  #global extencion    ####....................................................................... formas de elegir un área de interés
  extencion = None

  selected_value = opcion_area.value
  #print("Valor seleccionado:", selected_value)

  if (selected_value == 'Dibujar polígono en el mapa'):
  # ..................................................Región DIBUJADA
    Map.draw_features
    if (Map.draw_last_feature != None):
      extencion = ee.FeatureCollection(Map.draw_last_feature)#Roi Bounding Box (guarda en variable el bbox)
    #print("ok")

  if (selected_value == 'Utilizar carta 1:50.000'):
  # ..................................................Región CARTA
    if (lista_registros.value != None):
        valor_carta = lista_registros.value
        geometria_seleccionada = cartas.filter(ee.Filter.eq("carac", valor_carta))
        extencion = ee.FeatureCollection(geometria_seleccionada)

  return extencion

In [ ]:
#etiqueta_mensaje_clasif = widgets.Label(value=' ')

def clasificacion_agua_tierra(b):
  etiqueta_mensaje_clasif.value='...'

  global extencion
  extencion = area_extencion()

  if extencion is None:
    etiqueta_mensaje_clasif.value='Elija una carta o dibuje un poligono en el mapa'

  else:
    fecha = obtenerFecha()
    print(fecha)
    if fecha is not None:
      etiqueta_mensaje_clasif.value='Espere mientras se clasifica y se carga el resultado en el mapa'

      global classified_b  # Indica que deseas utilizar la variable global
####..............................................................................................................
    #extencion = area_extencion()
      try:
# .................................................. Colección de imágenes Sentinel 2
        #s2_b = ee.ImageCollection("COPERNICUS/S2") \
        s2_b = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
            .filterDate(*obtenerFecha()) \
            .filterBounds(extencion) \
            .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 0.2)

        s2c_b = s2_b.reduce(ee.Reducer.median())
        s2c_b = s2c_b.clip(extencion)

# NDWI
        NDWI_b = s2c_b.normalizedDifference(['B3_median', 'B8_median']).rename('NDWImedian')

# Stack
        stackIslas_b = NDWI_b

# Clasificar la imagen
        classified_b = stackIslas_b.classify(classifier)

# Añadir capa clasificada al mapa
        Map.addLayer(classified_b, {'min': 1, 'max': 2, 'palette': N1Color}, 'Clasificado')
        etiqueta_mensaje_clasif.value='Se ha cargado el resultado en el mapa'
    #print("Se ha cargado en el mapa el resultado de la clasificación")

      except ee.EEException as e:
        etiqueta_mensaje_clasif.value='Por favor, elija otro rango de fechas. Es posible que no haya datos válidos para ese período'




In [ ]:
#----------  Botón  ----------#
# Establece el ancho del botón
button_width = '130px'  # Puedes ajustar el valor según tus necesidades

submit = widgets.Button(
    description='CLASIFICAR',
    button_style='primary',
    style = widgets.ButtonStyle(button_color="#0072bb"),
    tooltip='Click en el botón para detectar agua-tierra',
    layout=widgets.Layout(width=button_width)
)

output_s = widgets.Output()

#si se presiona el botón se llama a la función
submit.on_click(clasificacion_agua_tierra)

In [ ]:
# Color de categorías Agua/NoAgua
N1Color = [
    '#0000ff',  # Corriente de agua - clase 1
    '#764C04'  # Tierra/NoAgua - clase 2
]

imageVisParam2 = {
    'bands': ["B3_median", "B2_median", "B1_median"],
    'gamma': 1,
    'max': 2789.728888888889,
    'min': 1875.3822222222223,
    'opacity': 1
}

In [ ]:
etiqueta_mensaje_r1 = widgets.Label(value='')
etiqueta_mensaje_r2 = widgets.Label(value='')

def descarga_raster(b):
    global classified_b  # Indica que deseas utilizar la variable global
    global extencion

    etiqueta_mensaje_r1.value = ''
    etiqueta_mensaje_r2.value = ''


    if classified_b is not None:
      etiqueta_mensaje_r1.value = 'Procesando descarga Raster. Por favor, espere.'

      try:
            url = classified_b.getDownloadURL({
                'name': 'LimiteAguaTierra',
                'scale': 10,
                'region': extencion.geometry()
            })

            etiqueta_mensaje_r2.value = f'Ingresar al siguiente link para descargar el resultado en formato Raster: {url}'
      except Exception as e:
            etiqueta_mensaje_r1.value = 'Error al descargar la imagen Raster. Por favor, elija un área mas pequeña.'
            #print(f'Error al descargar la imagen: {e}')

    else:
      etiqueta_mensaje_r1.value = 'Primero debe clasificar una zona'



In [ ]:
# Establece el ancho del botón
button_width_2 = '130px'  # Puedes ajustar el valor según tus necesidades

boton_descarga_raster = widgets.Button(
    description='Descarga Raster',
    button_style='primary',
    tooltip='Click en el botón para descargar el resultado en formato raster',
    style = widgets.ButtonStyle(button_color="#f9a822"),
    layout=widgets.Layout(width=button_width_2)
)

#si se presiona el botón se llama a la función
boton_descarga_raster.on_click(descarga_raster)

In [ ]:
import time

In [ ]:

def descarga_vectorial(b):
    global classified_b
    global extencion

    etiqueta_mensaje_r1.value = ''
    etiqueta_mensaje_r2.value = ''

    if classified_b is not None:
        etiqueta_mensaje_r1.value = 'Procesando descarga Vectorial. El archivo será descargado en Google Drive. Buscarlo como Clasificacion_Agua_Tierra. Tener en cuenta que puede demorar unos minutos.'

        try:
            # Muestra un mensaje de progreso
            with output:
                print('Procesando descarga Vectorial...')

            vectores = classified_b.reduceToVectors(
                reducer=ee.Reducer.countEvery(),
                scale=10,
                geometry=extencion
            )

            Agua = ee.FeatureCollection(vectores)

            descripcion = 'Clasificacion_Agua_Tierra'
            formato = 'SHP'

            exportacion = ee.batch.Export.table.toDrive(
                collection=Agua,
                description=descripcion,
                folder='',
                fileNamePrefix=descripcion,
                fileFormat=formato
            )

            # Iniciar la exportación
            exportacion.start()

            # Agregar una pausa antes de verificar el estado
            time.sleep(10)  # Espera 10 segundos (ajusta el tiempo según tus necesidades)

            # Verificar el estado de la exportación
            while exportacion.status()['state'] == 'RUNNING':
                time.sleep(1)  # Espera un segundo antes de verificar nuevamente

            if exportacion.status()['state'] == 'COMPLETED':
                etiqueta_mensaje_r1.value = 'Descarga Vectorial completada. El archivo está en Google Drive.'
            else:
                etiqueta_mensaje_r1.value = 'Error durante la exportación vectorial. Por favor, inténtelo de nuevo más tarde.'

        except Exception as e:
            etiqueta_mensaje_r1.value = 'Error durante la exportación vectorial. Por favor, inténtelo de nuevo más tarde.'
            print(f'Error durante la exportación vectorial: {e}')
    else:
        etiqueta_mensaje_r1.value = 'Primero debe clasificar una zona'






In [ ]:
#---------------------------------------------
boton_descarga_vectorial = widgets.Button(
    description='Descarga Vectorial',
    button_style='primary',
    tooltip='Click en el botón para descargar el resultado en formato vectorial',
    #font-size: 40px;
    style = widgets.ButtonStyle(button_color="#f9a822"),
    layout=widgets.Layout(width=button_width_2)
)

output = widgets.Output()

#si se presiona el botón se llama a la función
boton_descarga_vectorial.on_click(descarga_vectorial)

In [ ]:
Map

Map(center=[-40, -64], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [ ]:
etiqueta_desc = widgets.Label(value='(Las cartas se pueden ir visualizando en el mapa a medida que se seleccionan)')

# Colocar la etiqueta en una VBox
bb = widgets.VBox([etiqueta_desc])


#bot = widgets.VBox([submit, etiqueta_mensaje_clasif ])
# Configurar el margen entre A y B para separarlos
#bot.layout.margin = "0 0 0 120px"  # Márgenes en el formato "arriba derecha abajo izquierda"


vbox = widgets.VBox([etiqueta_linea, etiqueta_fecha, hbox_fecha, etiqueta_vacia, opcion_area, lista_registros,bb, etiqueta_vacia, submit, etiqueta_mensaje_clasif, etiqueta_vacia, etiqueta_linea])

# Configurar el margen entre A y B para separarlos
#vbox.layout.margin = "0 10px 0 100px"  # Márgenes en el formato "arriba derecha abajo izquierda"

# Mostrar el HBox
vbox


# Aplicar estilo al VBox principal para centrar el contenido
vbox.layout.align_items = 'center'  # Centra verticalmente el contenido de los VBox secundarios
#vbox.layout.justify_content = 'center'

# Aplicar estilo CSS para que el VBox principal ocupe el 100% del ancho de la pantalla
vbox_style = """
<style>
.vbox-main {
    width: 100%;
    max-width: 100%;
    font-size: 20px;
}
</style>
"""
widgets.VBox([widgets.HTML(vbox_style), vbox])

In [ ]:
etiqueta_desc = widgets.Label(value='El resultado obtenido puede ser descargado en los siguientes formatos: ')
A= widgets.VBox([ boton_descarga_raster])
B = widgets.VBox([ boton_descarga_vectorial])

# Configurar márgenes para los VBox A y B para separarlos
A.layout.margin = "0 0 0 50px"  # Márgenes a la derecha de A # Márgenes en el formato "arriba derecha abajo izquierda"
B.layout.margin = "0 0 0 50px"  # Márgenes a la izquierda de B

# Crear un HBox para organizar A y B horizontalmente
hbox_b2 = widgets.HBox([A, B])

# Mostrar el HBox
b_desc = widgets.VBox([etiqueta_desc, hbox_b2, etiqueta_mensaje_r1, etiqueta_mensaje_r2])

b_desc.layout.align_items = 'center'  # Centra verticalmente el contenido de los VBox secundarios

widgets.VBox([widgets.HTML(vbox_style), b_desc])

In [ ]:
# Organizar el mapa y las herramientas en un diseño de cuadrícula
grid_layout_v2 = widgets.GridBox([
    Map,  # Mapa en la columna 0
    b_desc
], layout=widgets.Layout(grid_template_columns='50% 50%'))

# Modificar el alto del GridBox
#grid_layout_v2.layout.height = '150px'  # Puedes ajustar el alto según lo necesites

# Ajustar la alineación del contenido hacia arriba
grid_layout_v2.layout.justify_content = 'flex-start'

display(grid_layout_v2)